## Environment setup


In [ ]:
%env DSP_CACHEBOOL=false

In [ ]:
import logging

logging.basicConfig(encoding="utf-8", level=logging.INFO)

In [ ]:
%run ./module-00-data-prep.ipynb
train = train
val = val
test = test
len(train), len(val), len(test)

## Model connection
### Models
* llama3.2:3b
* llama3.1:8b
* gemma2:9b

In [ ]:
import dspy

# MODEL = "llama3.2:3b-instruct-fp16"
# MODEL = "gemma2:2b"
# MODEL = "qwen2.5:0.5b"
# MODEL = "qwen2.5:1.5b"
# MODEL = "qwen2.5:3b"
# MODEL = "qwen2.5:7b"
# MODEL = "gemma2:9b"
MODEL = "llama3.1:8b"

lm = dspy.OllamaLocal(
    model=MODEL,
)
dspy.configure(lm=lm)

dspy.settings.configure(lm=lm)
dspy.configure(experimental=True)

In [ ]:
from sms_classifier import Input, Output


class SMSClassifierSignature(dspy.Signature):
    """
    Given an SMS text, predict whether it is ham, spam, or smishing.
    Output only the predicted label.
    """

    input: Input = dspy.InputField()
    output: Output = dspy.OutputField()

In [ ]:
class SMSClassifier(dspy.Module):
    def __init__(self, lm):
        self.lm = lm
        super().__init__()
        dspy.configure(lm=lm)
        self.generate_answer = dspy.TypedPredictor(
            SMSClassifierSignature, max_retries=5
        )

    def forward(self, input):
        return self.generate_answer(input=input)


sms_classifier = SMSClassifier(lm)

In [ ]:
from datetime import datetime

from evaluation_helpers import validate_answer
from langfuse_extensions import EvaluateWithLangfuse

session = f"Run-{MODEL}-{int(datetime.now().timestamp())}"
print(session)
# from evaluation_helpers import EvaluateWithLangfuse
evaluator = EvaluateWithLangfuse(
    devset=test, num_threads=1, display_progress=True, run_id=session
)
evaluator(sms_classifier, metric=validate_answer)

In [ ]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
from evaluation_helpers import validate_answer

config = dict(
    max_bootstrapped_demos=4,
    max_labeled_demos=8,
    num_candidate_programs=16,
    num_threads=1,
    max_errors=10,
)

teleprompter = BootstrapFewShotWithRandomSearch(metric=validate_answer, **config)
optimized_program = teleprompter.compile(sms_classifier, trainset=train[:100])

In [ ]:
session = f"Run-{MODEL}-optim-{int(datetime.now().timestamp())}"
print(session)
evaluator = EvaluateWithLangfuse(
    devset=test, num_threads=1, display_progress=True, run_id=session
)
evaluator(optimized_program, metric=validate_answer)

In [ ]:
optimized_program.save("programs/sms_classifier-qwen2.5-3b.json")